# Task #1: Get Info Box (store in Python Dictionary)

### Import Necessary Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

### Load the webpage

In [ ]:
# Load the webpage
r=requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')

# Convert to a beautiful Soup object

soup=bs(r.content)

# Print out the HTML content

contents=soup.prettify()
print(contents)


# IDEA: Be thinking about what you want to do and use Google to your advantage.

Let's  try to narrow down what we need:
 - The task we need right now is to get our info box

In [ ]:
info_box = soup.find(class_='infobox vevent')
print(info_box.prettify())

what do we get in the info box:
- first thing we see is the name.
- further down we get the director, producer.
- first things we want to do is strart to grab the table rows (tr) elements that contain the information.

In [ ]:
# we can use .find_all('tr') to get a list of all of the table rows.
info_rows_list = info_box.find_all("tr")
for row in info_rows_list:
    print(row.prettify())

In [ ]:
# create an empty dictionary named movie_info to store the information as key-value pairs.
# looks like the title 'Toy Story 3' is stored in the first <th> element, so we will use a .find()
movie_info = {}


# Since the information in the table rows may have different formats such as containing more than one name for a field.
# To know which <tr> items to handle diffently if we use enumerate(info_rows_list) it will give us an index for each item.
for index,row in enumerate(info_rows_list):
    print(index,row)



In [ ]:
# let's try getting our movie title. stored in the first item (index==0) of info_rows_list in the <th> element.
for index, row in enumerate(info_rows_list):
    if index==0:
        movie_info['title'] = row.find("th").get_text()
movie_info       

In [ ]:
# Trying to get more of the movie details using .get_text for each item in info_rows_list.
movie_details = []
for index, row in enumerate(info_rows_list):
    movie_details.append((index, row.get_text()))

movie_details

Some of the items have wierd characters in them. \
Can we manipulate those?

In [ ]:
# try getting rid of the new line character '\n'
movie_details[5][1].split('\n')

#This turns the string into a list.

In [ ]:
# creating a key value for movie_info dictionary that is the first item of the list, and storying the rest of the list as the value.
movie_info[movie_details[5][1].split('\n')[0]]=movie_details[5][1].split('\n')[1:]
movie_info

In [ ]:
# Try spliting the details by 'by' 
movie_details[2][1].split('by')

In [ ]:
### BELOW: Messing around with creating a dataframe



# movie_info[( movie_details[2][1].split('by')[0])] =  movie_details[2][1].split('by')[1:]
# movie_info

# movie_details[2][1].find('\n')

# ' '.join(movie_details[5][1].split())

# row_text = [''.join(x[1]) for x in movie_details]

# df = pd.DataFrame(columns = movie_info.keys())
# df

# df['title'] = movie_info['title']

# movie_info['title']

# df

# test = df.insert(0, column='title1', value = movie_info['title'])

# df

# pd.MultiIndex().from_arrays()

# arrays = [[1, 1, 2, 2], ['red', 'blue', 'red', 'blue']]
# pd.MultiIndex.from_arrays(arrays, names=('number', 'color'))

# MultiIndex.to_frame()



# keys = np.array(movie_info.keys())

# array = np.array(list(movie_info.values()))





# pd.MultiIndex.from_arrays(array)

Let's go back to building the dictionary using the for loop and indexes to help. \
Each of the column headers (keys) are stored in the table head `<th>` elements of the table rows `<tr>`. \
The content values (values) are stored in the table details `<td>` elements of the table rows `<tr>`. \
NOTE: index==1 is the info box poster, and we dont need that index, so after index==0 we can start with index > 1.

In [ ]:
for index, row in enumerate(info_rows_list):
    if index==0:
        movie_info['title'] = row.find("th").get_text()
    elif index > 1:
        content_key = row.find('th').get_text()
        content_value = row.find('td').get_text()
        movie_info[content_key] = content_value
        
movie_info       

Looks pretty good except for some of the special characters... \
\
The special characters look like they start at 'Story by'. Let's go back to that item in the html to see how these names are handled. \ 
- Looks like if there is more than one name then inside the `<td>` element there is an unordered list `<ul>`, and inside of the `<ul>` are the line items `<li>` that contains the names. 
- Let's create separate 'if' statement that can handle these indexes.

In [ ]:
# Testing if statement
for index,row in enumerate(info_rows_list):
    # if the index > 1 and row contains an unordered list:
    if index > 1 and row.find('ul'):
        # get the content_key
        content_key = row.find('th').get_text()
        # content value is going to be in all <li> elements.
        content_value = row.find_all('li')
        # get the lext from all the elements, and store in a list
        content_value = [x.get_text() for x in content_value] 
        movie_info[content_key] = content_value
        
movie_info

Let's make a function to contain the action that happens after the 'if' statement

### function: get_content_key_and value(row_data):

In [ ]:
# takes in row_data to find the table head <th>, and get_text() to store as a variable 'content_key'
# if row_data contains list items <li>, then it will find all <li> elements and store as a list 'li_list'
# for each item in 'li_list', .get_text() and store into a list 'content_value'
# else it will find the <td> elements and get_text() to store as a variable 'content_value'
# Returns a dictionary object {content_key, content_value}

def get_content_key_and_value(row_data):
        content_key = row_data.find('th').get_text()
        # content value is going to be in all <li> elements.
        if row_data.find('li'):
            li_list = row_data.find_all('li')
            # get the lext from all the elements, and store in a list
            content_value = [x.get_text() for x in li_list] 
        else:
            content_value = row_data.find('td').get_text()
        return {content_key: content_value}
            
            

In [ ]:
# Testing function get_content_key_and_value(row_data)
update_key = get_content_key_and_value(info_rows_list[5])
update_key

In [ ]:
movie_info.update(update_key)

In [ ]:
movie_info

In [ ]:
# Try the function with the for loop again to get rid of '\n' in movie_info_dictionary.
# reset movie_info dictionary
movie_info = {}
for index, row in enumerate(info_rows_list):
    if index==0:
        movie_info['title'] = row.find("th").get_text()
    elif index > 1:
        update_dictionary = get_content_key_and_value(row)
        movie_info.update(update_dictionary)
        

movie_info
        

Looks like there are some things to still clean up.
- Production Company key looks like one word 'ProductionCompany'.
- in the Release date and Box office keys there is a weird `\xao` character.
    - This could be updated by replacing `\xao` with a '-' or space.

In [ ]:
# starting with the string replacement because it seams to be more of a low hanging fruit.
# start with looking at the key 'Release date'
movie_info['Release date']

In [ ]:
[x.find('\xa0') for x in movie_info['Release date']]

In [ ]:
[x.replace('\xa0', ' ') for x in movie_info['Release date'] if x.find('\xa0')]

In [ ]:
# let's do .replace('\xa0', ' ') in the get_content_key_and_value(row_data)
def get_content_key_and_value(row_data):
        content_key = row_data.find('th').get_text()
        # content value is going to be in all <li> elements.
        if row_data.find('li'):
            li_list = row_data.find_all('li')
            # get the lext from all the elements, and store in a list
            content_value = [x.get_text().replace('\xa0', ' ') for x in li_list] 
        else:
            content_value = row_data.find('td').get_text().replace('\xa0', ' ')
        return {content_key: content_value}

In [ ]:
# Try the function with the for loop again to get rid of '\xa0' in movie_info_dictionary.
# reset movie_info dictionary
movie_info = {}
for index, row in enumerate(info_rows_list):
    if index==0:
        movie_info['title'] = row.find("th").get_text()
    elif index > 1:
        update_dictionary = get_content_key_and_value(row)
        movie_info.update(update_dictionary)
        

movie_info
        

In [ ]:
# Now dealing with 'Productioncompany' key and StudiosMotion Pictures.
# In the Beautiful Soup Documentation with the .get_text() method you can specify separator character such as .get_text("/")
# Also, with .get_text() you can strip the white space with strip=True.
# Let's use these paramenters with .get_text() to see if we can separate the values.

# we will update the function: get_content_key_and_value(row_data)

def get_content_key_and_value(row_data):
        content_key = row_data.find('th').get_text(" ", strip=True)
        # content value is going to be in all <li> elements.
        if row_data.find('li'):
            li_list = row_data.find_all('li')
            # get the lext from all the elements, and store in a list
            content_value = [x.get_text(" ", strip=True).replace('\xa0', ' ') for x in li_list] 
        else:
            content_value = row_data.find('td').get_text(" ", strip=True).replace('\xa0', ' ')
        return {content_key: content_value}

In [ ]:
# Try the function with the for loop again to use the .get_text() method with parameters (" ", strip=True).
# reset movie_info dictionary
movie_info = {}
for index, row in enumerate(info_rows_list):
    if index==0:
        movie_info['title'] = row.find("th").get_text()
    elif index > 1:
        update_dictionary = get_content_key_and_value(row)
        movie_info.update(update_dictionary)
        

movie_info
        

# Task 2:

- [ ] Scrape infobox for all movies in List of Disney Films (save as list of dictionaries)
- LINK: [List of Disney Films](https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films) 

In [2]:
# Load the webpage:
# Load the webpage
r=requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# Convert to a beautiful Soup object

soup=bs(r.content)


In [3]:
titles = soup.select('table.wikitable.sortable i a')
titles

[<a href="/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons" title="Academy Award Review of Walt Disney Cartoons">Academy Award Review of Walt Disney Cartoons</a>,
 <a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a>,
 <a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a>,
 <a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a>,
 <a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a>,
 <a href="/wiki/Dumbo" title="Dumbo">Dumbo</a>,
 <a href="/wiki/Bambi" title="Bambi">Bambi</a>,
 <a href="/wiki/Saludos_Amigos" title="Saludos Amigos">Saludos Amigos</a>,
 <a href="/wiki/Victory_Through_Air_Power_(film)" title="Victory Through Air Power (film)">Victory Through Air Power</a>,
 <a href="/wiki/The_Three_Caballeros" title="The Three Caballeros">The Three Caballeros</a>,
 <a href="/wiki/M

In [7]:
movies = [x.get_text() for x in titles]
movies

['Academy Award Review of Walt Disney Cartoons',
 'Snow White and the Seven Dwarfs',
 'Pinocchio',
 'Fantasia',
 'The Reluctant Dragon',
 'Dumbo',
 'Bambi',
 'Saludos Amigos',
 'Victory Through Air Power',
 'The Three Caballeros',
 'Make Mine Music',
 'Song of the South',
 'Fun and Fancy Free',
 'Melody Time',
 'So Dear to My Heart',
 'The Adventures of Ichabod and Mr. Toad',
 'Cinderella',
 'Treasure Island',
 'Alice in Wonderland',
 'The Story of Robin Hood and His Merrie Men',
 'Peter Pan',
 'The Sword and the Rose',
 'The Living Desert',
 'Rob Roy, the Highland Rogue',
 'The Vanishing Prairie',
 '20,000 Leagues Under the Sea',
 'Davy Crockett, King of the Wild Frontier',
 'Lady and the Tramp',
 'The African Lion',
 'The Littlest Outlaw',
 'The Great Locomotive Chase',
 'Davy Crockett and the River Pirates',
 'Secrets of Life',
 'Westward Ho the Wagons!',
 'Johnny Tremain',
 'Perri',
 'Old Yeller',
 'The Light in the Forest',
 'White Wilderness',
 'Tonka',
 'Sleeping Beauty',
 'The 

In [10]:
[(index, movie) for index, movie in enumerate(movies) if movie=="The Nightmare Before Christmas 3D"]

[(301, 'The Nightmare Before Christmas 3D')]

In [11]:
titles[301]['href']

'/wiki/The_Nightmare_Before_Christmas'

In [12]:
movie_pages = [x['href'] for x in titles]
movie_pages

['/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons',
 '/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)',
 '/wiki/Pinocchio_(1940_film)',
 '/wiki/Fantasia_(1940_film)',
 '/wiki/The_Reluctant_Dragon_(1941_film)',
 '/wiki/Dumbo',
 '/wiki/Bambi',
 '/wiki/Saludos_Amigos',
 '/wiki/Victory_Through_Air_Power_(film)',
 '/wiki/The_Three_Caballeros',
 '/wiki/Make_Mine_Music',
 '/wiki/Song_of_the_South',
 '/wiki/Fun_and_Fancy_Free',
 '/wiki/Melody_Time',
 '/wiki/So_Dear_to_My_Heart',
 '/wiki/The_Adventures_of_Ichabod_and_Mr._Toad',
 '/wiki/Cinderella_(1950_film)',
 '/wiki/Treasure_Island_(1950_film)',
 '/wiki/Alice_in_Wonderland_(1951_film)',
 '/wiki/The_Story_of_Robin_Hood_and_His_Merrie_Men',
 '/wiki/Peter_Pan_(1953_film)',
 '/wiki/The_Sword_and_the_Rose',
 '/wiki/The_Living_Desert',
 '/wiki/Rob_Roy,_the_Highland_Rogue',
 '/wiki/The_Vanishing_Prairie',
 '/wiki/20,000_Leagues_Under_the_Sea_(1954_film)',
 '/wiki/Davy_Crockett,_King_of_the_Wild_Frontier',
 '/wiki/Lady_and_the_Tramp',
 '/wiki/Th

In [13]:
base_url = "https://en.wikipedia.org"

websites = [f'{base_url}{x}' for x in movie_pages]
websites

['https://en.wikipedia.org/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons',
 'https://en.wikipedia.org/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)',
 'https://en.wikipedia.org/wiki/Pinocchio_(1940_film)',
 'https://en.wikipedia.org/wiki/Fantasia_(1940_film)',
 'https://en.wikipedia.org/wiki/The_Reluctant_Dragon_(1941_film)',
 'https://en.wikipedia.org/wiki/Dumbo',
 'https://en.wikipedia.org/wiki/Bambi',
 'https://en.wikipedia.org/wiki/Saludos_Amigos',
 'https://en.wikipedia.org/wiki/Victory_Through_Air_Power_(film)',
 'https://en.wikipedia.org/wiki/The_Three_Caballeros',
 'https://en.wikipedia.org/wiki/Make_Mine_Music',
 'https://en.wikipedia.org/wiki/Song_of_the_South',
 'https://en.wikipedia.org/wiki/Fun_and_Fancy_Free',
 'https://en.wikipedia.org/wiki/Melody_Time',
 'https://en.wikipedia.org/wiki/So_Dear_to_My_Heart',
 'https://en.wikipedia.org/wiki/The_Adventures_of_Ichabod_and_Mr._Toad',
 'https://en.wikipedia.org/wiki/Cinderella_(1950_film)',
 'https://en.wikipedia.org/wi

In [14]:
len(websites)

449

In [15]:
len(movies)

449

There are 449 webpages we will go to!

In [16]:
request_list=[]

request_list = [requests.get(x) for x in websites]

# Convert to a beautiful Soup object

# soup=bs(r.content)
request_list

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [

In [17]:
# check that all of the requests were 200. That means a successful request.
len(['success' for x in request_list if x.status_code==200])

449

In [18]:
soups = [bs(x.content) for x in request_list]

In [19]:
info_boxes = [x.find(class_='infobox vevent') for x in soups]

In [20]:
type(info_boxes)

list

In [21]:
# write this list to a file to cache it
with open('movie_info_boxes.txt', 'w') as f:
    for item in info_boxes:
        f.write("%s\n" % item)

In [22]:
# did we get all the movies? If we captured all the movies the length of info boxes should be 449
len(info_boxes)

449

The output ^^^ says length is 449!!! Nice!!

In [24]:
# revisting the <tr> elements to iteriate through all of the table rows
movie = info_boxes[0]
movie

info_rows_list=movie.find_all('tr')

for index,row in enumerate(info_rows_list):
    print(index, row)

0 <tr><th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size:110%;font-style:italic;">Academy Award Review of <span class="nowrap">Walt Disney Cartoons</span></th></tr>
1 <tr><td colspan="2" style="text-align:center"><a class="image" href="/wiki/File:Aarfvdsf.jpg"><img alt="Aarfvdsf.jpg" class="thumbborder" data-file-height="111" data-file-width="300" decoding="async" height="81" src="//upload.wikimedia.org/wikipedia/en/thumb/7/7a/Aarfvdsf.jpg/220px-Aarfvdsf.jpg" srcset="//upload.wikimedia.org/wikipedia/en/7/7a/Aarfvdsf.jpg 1.5x" width="220"/></a><div style="font-size:95%;padding:0.35em 0.35em 0.25em;line-height:1.25em;">Opening and closing screens</div></td></tr>
2 <tr><th scope="row" style="white-space:nowrap;padding-right:0.65em;"><div style="display:inline-block; padding:0.1em 0;line-height:1.2em;">Production<br/>company </div></th><td><div style="vertical-align:middle;"><a href="/wiki/Walt_Disney_Animation_Studios" title="Walt Disney Ani

In [25]:
# use the for loop we did earlier to create our movie_info dictionaries.
movie_dictionaries_list=[]
count = 0
for movie in info_boxes:
    movie_info = {}
    info_rows_list=movie.find_all('tr')
    for index, row in enumerate(info_rows_list):
        if index==0:
            movie_info['title'] = row.find("th").get_text()
        elif (index > 1) and get_content_key_and_value(row) is None:
            update_dictionary = get_content_key_and_value(row)
            movie_info.update(update_dictionary)
            movie_dictionaries_list.append(movie_info)
    print(count,movie_info['title'])
    count += 1

NameError: name 'get_content_key_and_value' is not defined

Exploration into the series Zorro is below, and the updated function follows.

In [ ]:
# info_rows_list=info_boxes[43].find_all('tr')
# [print(index,row) for index, row in enumerate(info_rows_list)]

In [ ]:
# movie_info = {}
# info_rows_list=info_boxes[43].find_all('tr')
# for index, row in enumerate(info_rows_list):
#     if index==0:
#         movie_info['title'] = row.find("th").get_text()
#         print(movie_info['title'])
#     elif (index > 1):
#         update_dictionary = get_content_key_and_value(row)
#         movie_info.update(update_dictionary)
#     print(index, update_dictionary)

In [ ]:
# row_data = info_rows_list[20]
# row_data

In [ ]:
# content_key = row_data.find('th').get_text(" ", strip=True)
# content_key

In [ ]:
# row_data.find('li') is not None

In [ ]:
# def skip_content(row_data):
#         if row_data.find('th') is not None:
#             content_key = row_data.find('th').get_text(" ", strip=True)
#         else:
#             content_key = 'No_content_key'
#         # content value is going to be in all <li> elements.
#         if row_data.find('li') is not None:
#             li_list = row_data.find_all('li')
#             # get the lext from all the elements, and store in a list
#             content_value = [x.get_text(" ", strip=True).replace('\xa0', ' ') for x in li_list] 
#         elif row_data.find('td') is not None:
#             content_value = row_data.find('td').get_text(" ", strip=True).replace('\xa0', ' ')
#         else:
#             return {content_key: 'No Content'}
#         return {content_key: content_value}
    

In [ ]:
# skip_content(row_data)

In [ ]:
# # try the dictionary again with the skip_content(row_data) function instead of get_content_key_and_value(row_data)

# movie_info = {}
# info_rows_list=info_boxes[43].find_all('tr')
# for index, row in enumerate(info_rows_list):
#     if index==0:
#         movie_info['title'] = row.find("th").get_text()
#     elif (index > 1):
#         update_dictionary = skip_content(row)
#         movie_info.update(update_dictionary)
#     print(index, update_dictionary)

In [ ]:
# movie_info

# UPDATE FUNCTION: get_content_key_and_value(row_data)

we saw from above with Zorrow that this is a series and not structured like all the other movie boxes. We put conditions into the updated function below to handle this use case and keep using the function.

In [26]:
def get_content_key_and_value(row_data):
        if row_data.find('th') is not None:
            content_key = row_data.find('th').get_text(" ", strip=True)
        else:
            content_key = 'No_content_key'
        # content value is going to be in all <li> elements.
        if row_data.find('li') is not None:
            li_list = row_data.find_all('li')
            # get the lext from all the elements, and store in a list
            content_value = [x.get_text(" ", strip=True).replace('\xa0', ' ') for x in li_list] 
        elif row_data.find('td') is not None:
            content_value = row_data.find('td').get_text(" ", strip=True).replace('\xa0', ' ')
        else:
            return {content_key: 'No Content'}
        return {content_key: content_value}
    

# Make a List of the Dictionaries

In [27]:
movie_dictionaries_list=[]
count = 0
for movie in info_boxes:
    movie_info = {}
    if movie is not None:
        info_rows_list=movie.find_all('tr')
        for index, row in enumerate(info_rows_list):
            if index==0 and row.find('th') is not None:
                movie_info['title'] = row.find("th").get_text()
            elif (index > 1):
                update_dictionary = get_content_key_and_value(row)
                movie_info.update(update_dictionary)
    else:
        movie_info['title']='NO MOVIE INFORMATION'
    
    movie_dictionaries_list.append(movie_info)

    print(count,movie_info['title'])
    count += 1

0 Academy Award Review of Walt Disney Cartoons
1 Snow White and the Seven Dwarfs
2 Pinocchio
3 Fantasia
4 The Reluctant Dragon
5 Dumbo
6 Bambi
7 Saludos Amigos
8 Victory Through Air Power
9 The Three Caballeros
10 Make Mine Music
11 Song of the South
12 Fun and Fancy Free
13 Melody Time
14 So Dear to My Heart
15 The Adventures of Ichabod andMr. Toad
16 Cinderella
17 Treasure Island
18 Alice in Wonderland
19 The Story of Robin Hood andHis Merrie Men
20 Peter Pan
21 The Sword and the Rose
22 The Living Desert
23 Rob Roy, the Highland Rogue
24 The Vanishing Prairie
25 20,000 Leagues Under the Sea
26 Davy Crockett, King of the Wild Frontier
27 Lady and the Tramp
28 The African Lion
29 The Littlest Outlaw
30 The Great Locomotive Chase
31 Davy Crockett and the River Pirates
32 Secrets of Life
33 Westward Ho the Wagons!
34 Johnny Tremain
35 Perri
36 Old Yeller
37 The Light in the Forest
38 White Wilderness
39 Tonka
40 Sleeping Beauty
41 The Shaggy Dog
42 Darby O'Gill and the Little People
43 

369 Planes
370 Frozen
371 Saving Mr. Banks
372 Muppets Most Wanted
373 Bears
374 Million Dollar Arm
375 Maleficent
376 Planes: Fire & Rescue
377 Khoobsurat
378 Alexander and the Terrible, Horrible, No Good, Very Bad Day
379 Big Hero 6
380 Into the Woods
381 McFarland, USA
382 Cinderella
383 Monkey Kingdom
384 Tomorrowland
385 Inside Out
386 ABCD 2
387 The Good Dinosaur
388 The Finest Hours
389 Zootopia
390 The Jungle Book
391 Tini: The Movie
392 Alice Through the Looking Glass
393 Finding Dory
394 The BFG
395 Pete's Dragon
396 Queen of Katwe
397 Moana
398 Dangal
399 March of the Penguins
400 Beauty and the Beast
401 Born in China
402 Pirates of the Caribbean:Dead Men Tell No Tales
403 Cars 3
404 Ghost of the Mountains
405 Jagga Jasoos
406 Coco
407 A Wrinkle in Time
408 Incredibles 2
409 Christopher Robin
410 The Nutcracker and the Four Realms
411 Ralph Breaks the Internet
412 Mary Poppins Returns
413 Dumbo
414 Penguins
415 Aladdin
416 Toy Story 4
417 The Lion King
418 Maleficent: Mistr

In [28]:
type(info_boxes[124])

NoneType

In [29]:
info_boxes[440]

In [30]:
movie_dictionaries_list[0]['title'], movie_dictionaries_list[1]['title']

('Academy Award Review of Walt Disney Cartoons',
 'Snow White and the Seven Dwarfs')

In [31]:
# Find the movies and indexes in the list where the title is 'NO MOVIE INFORMATION'
no_movie_list = [(index, movie_info) for index, movie_info in enumerate(movie_dictionaries_list) if movie_info['title']=='NO MOVIE INFORMATION']
no_movie_list

[(124, {'title': 'NO MOVIE INFORMATION'}),
 (440, {'title': 'NO MOVIE INFORMATION'})]

Index 124: There was another <i> element in the row for 'The Best of Walt Disney's True-Life Adventures' This grabbed the href for the element that goes to a wiki page about the document series, but does not contain a movie info box
\
Index 440: This is for a movie named Luca that is an upcoming movie, right now the wiki link goes to the Pixar wiki page.
\
\
We will go ahead and delete these dictionaries from the list.

In [32]:
no_movie_indexes = [no_movie[0] for no_movie in no_movie_list]
no_movie_indexes

[124, 440]

In [33]:
movie_dictionaries_list = [movie_dictionaries_list[x] for x in range(len(movie_dictionaries_list)) if x not in no_movie_indexes]

We saw that 'Zorro' was a series, let's look at the dictionary and see if there are others like it.

In [34]:
[movie for movie in movie_dictionaries_list if movie['title']=='Zorro']

[{'title': 'Zorro',
  'Genre': ['Action / Adventure', 'Comedy-drama', 'Western', 'Superhero'],
  'Created by': 'Johnston McCulley (original stories)',
  'Starring': ['Guy Williams [1]',
   'Gene Sheldon',
   'Henry Calvin',
   'George J. Lewis'],
  'Country of origin': 'United States',
  'No. of seasons': '2',
  'No. of episodes': '78 (+4 specials) ( list of episodes )',
  'Production': 'No Content',
  'Executive producers': ['Bill Anderson', 'Walt Disney'],
  'Running time': '22–24 minutes',
  'Production company': 'Walt Disney Productions',
  'Release': 'No Content',
  'Original network': 'ABC',
  'Picture format': 'Black-and-white (also colorized )',
  'Audio format': 'Monaural',
  'Original release': 'October 10, 1957 ( 1957-10-10 ) – July 2, 1959 ( 1959-07-02 )',
  'Chronology': 'No Content',
  'Related shows': 'Zorro and Son',
  'External links': 'No Content',
  'No_content_key': 'Website'},
 {'title': 'Zorro',
  'Genre': ['Action / Adventure', 'Comedy-drama', 'Western', 'Superhe

There are two Zorro dictionaries! Since these are not theatrical releases let's get rid of these too, and find other dictionaries that may have 'No. of episodes' key.

In [35]:
series_list = [(index,movie) for index,movie in enumerate(movie_dictionaries_list) if 'No. of episodes' in movie]
series_list

[(43,
  {'title': 'Zorro',
   'Genre': ['Action / Adventure', 'Comedy-drama', 'Western', 'Superhero'],
   'Created by': 'Johnston McCulley (original stories)',
   'Starring': ['Guy Williams [1]',
    'Gene Sheldon',
    'Henry Calvin',
    'George J. Lewis'],
   'Country of origin': 'United States',
   'No. of seasons': '2',
   'No. of episodes': '78 (+4 specials) ( list of episodes )',
   'Production': 'No Content',
   'Executive producers': ['Bill Anderson', 'Walt Disney'],
   'Running time': '22–24 minutes',
   'Production company': 'Walt Disney Productions',
   'Release': 'No Content',
   'Original network': 'ABC',
   'Picture format': 'Black-and-white (also colorized )',
   'Audio format': 'Monaural',
   'Original release': 'October 10, 1957 ( 1957-10-10 ) – July 2, 1959 ( 1959-07-02 )',
   'Chronology': 'No Content',
   'Related shows': 'Zorro and Son',
   'External links': 'No Content',
   'No_content_key': 'Website'}),
 (48,
  {'title': 'Zorro',
   'Genre': ['Action / Adventure

These are the only movies that have an episodes field so we will get rid of these

In [36]:
series_list_indexes = [series[0] for series in series_list]


In [37]:
movie_dictionaries_list = [movie_dictionaries_list[x] for x in range(len(movie_dictionaries_list)) if x not in series_list_indexes]
movie_dictionaries_list

[{'title': 'Academy Award Review of Walt Disney Cartoons',
  'Production company': 'Walt Disney Productions',
  'Release date': ['May 19, 1937 ( 1937-05-19 )'],
  'Running time': '41 minutes (74 minutes 1966 release)',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$45.472'},
 {'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand (supervising)',
   'William Cottrell',
   'Wilfred Jackson',
   'Larry Morey',
   'Perce Pearce',
   'Ben Sharpsteen'],
  'Produced by': 'Walt Disney',
  'Written by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': 'Snow White by The Brothers Grimm',
  'Starring': ['Adriana Caselotti',
   'Lucille La Verne',
   'Harry Stockwell',
   'Roy Atwell',
   'Pinto Colvig',
   'Otis Harlan',
   'Scotty Mattraw',
   'Billy Gilbert',
   'Eddie Collins',
   'Moroni Olsen',
   'Stuart Buchanan'],
  'Music 

In [38]:
len(movie_dictionaries_list)

445

since we got rid of 4 total movies getting a length of 445 makes sense!